In [ ]:
import os
import pandas as pd
from astropy import units as u
from gammapy.estimators import FluxPoints
from gammapy.datasets import FluxPointsDataset
from gammapy.modeling import Fit
from gammapy.modeling.models import ExpCutoffPowerLawSpectralModel, PowerLawSpectralModel, SkyModel


folder_path = '3PC_SEDs/'
output_csv = 'pulsar_analysis_results.csv'


ecvs_files = [f for f in os.listdir(folder_path) if f.endswith('.ecsv')]

# List for results
results = []


for ecsv_file in ecvs_files:
    try:
        pulsar_name = os.path.splitext(ecsv_file)[0]  # get the pulsar name from ecsv files
        file_path = os.path.join(folder_path, ecsv_file)

        
        flux_points = FluxPoints.read(file_path)

        

        # PL model
        model_pl = PowerLawSpectralModel(
            amplitude=1e-8 * u.Unit("cm-2 s-1 GeV-1"),
            index=2.0,
            reference=1.0 * u.GeV
        )

        # ECPL model
        model_ecpl = ExpCutoffPowerLawSpectralModel(
            amplitude=1e-8 * u.Unit("cm-2 s-1 GeV-1"),
            index=2.0,
            lambda_=0.1 * u.Unit("GeV-1"),
            reference=1.0 * u.GeV
        )

        # Create SkyModel and dataset for PL
        fit_model_pl = SkyModel(spectral_model=model_pl)
        dataset_pl = FluxPointsDataset(fit_model_pl, flux_points)

        # Create SkyModel and dataset for ECPL
        fit_model_ecpl = SkyModel(spectral_model=model_ecpl)
        dataset_ecpl = FluxPointsDataset(fit_model_ecpl, flux_points)

        # Fit the PL model
        minuit_opts = {"tol": 0.001, "strategy": 2}
        fitter = Fit()
        fitter.optimize_opts = minuit_opts
        result_pl = fitter.run([dataset_pl])
        result_pl = fitter.run([dataset_pl])
        result_pl = fitter.run([dataset_pl])

        # Fit the ECPL model
        result_ecpl = fitter.run([dataset_ecpl])
        result_ecpl = fitter.run([dataset_ecpl])
        result_ecpl = fitter.run([dataset_ecpl])

        if result_ecpl.success:
            # Calculate e_cut and TS
            e_cut = 1 / model_ecpl.lambda_.value  
            ts_value = result_pl.total_stat - result_ecpl.total_stat  

            
            results.append({
                'pulsar_name': pulsar_name,
                'e_cut (GeV)': e_cut,
                'TS': ts_value
            })
        else:
            print(f"ECPL fit is unseccessfull: {pulsar_name}")

    except Exception as e:
        print(f"Error: ({ecsv_file}): {e}")

# Write outputs to a CSV file
results_df = pd.DataFrame(results)
results_df.to_csv(output_csv, index=False)

print(f"Results saved to file {output_csv} ")